# Web Scraping for Data

In [ ]:
# import pandas & numpy library
import pandas as pd
import numpy as np

# Import seaborn and apply its plotting styles
import seaborn as sns
sns.set(font_scale=2, style="white")

# import matplotlib
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
# set plotting size parameter
plt.rcParams['figure.figsize'] = (12, 5)

# packages helpful for webscraping
import requests
import bs4
from bs4 import BeautifulSoup
import re

#improve resolution
%config InlineBackend.figure_format ='retina'

# Web Scrape Wikipedia for Movie Plots, Genre, and Other Info
https://en.wikipedia.org/wiki/Category:American_films_by_genre

In [265]:
genre_link = {
#     'Action': [
#         'https://en.wikipedia.org/w/index.php?title=Category:American_action_films&pageuntil=Driving+Force+%281989+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_action_films&pagefrom=Driving+Force+%281989+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_action_films&pagefrom=Marksman%2C+The%0AThe+Marksman+%282005+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_action_films&pagefrom=Spider-Man+3#mw-pages'
#     ]
#     ,
#     'Crime': [
#         'https://en.wikipedia.org/wiki/Category:American_crime_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_crime_films&pagefrom=Dial+Red+O#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_crime_films&pagefrom=Ivy+%28Film%29%0AIvy+%281947+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_crime_films&pagefrom=One+Stolen+Night+%281929+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_crime_films&pagefrom=Tenderloin+%28film%29#mw-pages'
#     ]
#     ,
#     'War': [
#         'https://en.wikipedia.org/wiki/Category:American_war_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_war_films&pagefrom=Retreat%2C+Hell%21#mw-pages'
#     ]
#     ,
#     'Romance': [
#         'https://en.wikipedia.org/wiki/Category:American_romance_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_romance_films&pagefrom=Sporting+Venus%2C+The%0AThe+Sporting+Venus#mw-pages'
#     ]
#     ,
#     'Thriller': [
#         'https://en.wikipedia.org/wiki/Category:American_thriller_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_thriller_films&pagefrom=Godsend+%282004+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_thriller_films&pagefrom=Poltergeist+%28film%29%0APoltergeist+%281982+film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_thriller_films&pagefrom=Winchester+%28film%29#mw-pages'
#     ]
#     ,
#     'Horror': [
#         'https://en.wikipedia.org/wiki/Category:American_horror_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_horror_films&pagefrom=Isle+of+the+Dead+%28film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_horror_films&pagefrom=West+of+Hell#mw-pages'
#     ]
#     ,
#     'Biographical': [
#         'https://en.wikipedia.org/wiki/Category:American_biographical_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_biographical_films&pagefrom=I+Wanna+Dance+with+Somebody+%28film%29#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_biographical_films&pagefrom=Story+of+Alexander+Graham+Bell%0AThe+Story+of+Alexander+Graham+Bell#mw-pages'
#     ]
#     ,
#     'Satirical': [
#         'https://en.wikipedia.org/wiki/Category:American_satirical_films',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_satirical_films&pagefrom=Hospital%2C+The%0AThe+Hospital#mw-pages',
#         'https://en.wikipedia.org/w/index.php?title=Category:American_satirical_films&pagefrom=Taintlight%0ATaintlight#mw-pages'
#     ]
#     ,
    'Splatter': [
        'https://en.wikipedia.org/wiki/Category:American_splatter_films',
        'https://en.wikipedia.org/w/index.php?title=Category:American_splatter_films&pagefrom=Night+of+the+Demon+%281980+film%29#mw-pages'
    ]
    ,
    'Science Fiction': [
        'https://en.wikipedia.org/wiki/Category:American_science_fiction_films',
        'https://en.wikipedia.org/w/index.php?title=Category:American_Western_(genre)_films&pagefrom=Big+Sombrero%2C+The%0AThe+Big+Sombrero+%28film%29#mw-pages'
    ]
    ,
    'Monster': [
        'https://en.wikipedia.org/wiki/Category:American_monster_movies',
        'https://en.wikipedia.org/w/index.php?title=Category:American_monster_movies&pagefrom=Fly%2C+The%0AThe+Fly+%281986+film%29#mw-pages',
        'https://en.wikipedia.org/w/index.php?title=Category:American_monster_movies&pagefrom=Nailbiter#mw-pages',
        'https://en.wikipedia.org/w/index.php?title=Category:American_monster_movies&pagefrom=World+Without+End+%28film%29#mw-pages'
    ]
}

In [258]:
info_list = [
    'Directed by',
    'Written by',
    'Story by',
    'Produced by',
    'Starring',
    'Cinematography',
    'Edited by',
    'Music by',
    'Production company',
    'Distributed by',
    'Release date',
    'Running time',
    'Budget',
    'Box office'
]

In [347]:
%%time

# Create a blank dataframe and assign column names
scraped_df = pd.DataFrame(columns=['Film name', 'Genre', 'Plot'] + info_list)

# Loop through the genres
# category_links is a list of URLs for the same genre, but each URL contains around 200 individual films
for genre, category_links in genre_link.items():
    
    # Each movie genre has several URLs since not everything can fit on the same page
    # Loop through the URLs in category_links to access all the URLs for each movie genre
    for category_link in category_links:
        req = requests.get(category_link, timeout=5)
        soup = BeautifulSoup(req.content, 'html') # get contents of web page    
        
        soup.find('div', {'class': 'mw-content-ltr'}).find_all('a', href=True)
        
        # Each category_link has around 200 individual films' URLs
        # This for loop populates name_links dictionary with the film name as key and Wikipedia page URL as value
        name_links = {}
        for a in soup.find('div', {'class': 'mw-content-ltr'}).find_all('a', href=True):
            film_name = a.text
            link_end = a['href']
            link = 'https://en.wikipedia.org' + link_end
            if 'Categor' not in link: # skip if 'Category' or 'Categorization' is in the link
                name_links[film_name] = link
        
        # Loop through the individual films' URLs to extract wanted info
        for film_name, link in name_links.items():
            req = requests.get(link, timeout=5)
            soup = BeautifulSoup(req.content, 'html') # get contents of web page
            tag_contents = soup.select('p, span.mw-headline')
            
            # Extract the plot (can also be named "Premise" or "Synopsis") if it exists
            start_index_of_plot = -1
            end_index_of_plot = -1
            plot_exists = False

            for i, tag_content in enumerate(tag_contents):
                tag = tag_content.name
                content = tag_content.text

                if (plot_exists) & (tag == 'span'):
                    end_index_of_plot = i
                    break

                if (content == 'Plot') | (content == 'Premise') | (content == 'Synopsis'):
                    start_index_of_plot = i + 1
                    plot_exists = True

            plot = []
            for content in tag_contents[start_index_of_plot: end_index_of_plot]:
                par = content.text
                plot += [par]

            plot = "".join(plot).strip()
            
            # If plot exists, extract other info as well
            if len(plot) >= 1:
                
                film_dict = {'Film name': film_name, 'Genre': genre, 'Plot': plot}
                
                # Get other info on the film, if the info table eixsts 
                try:
                    req = requests.get(link, timeout=5)
                    soup = BeautifulSoup(req.content, 'html')
                    wiki_tables = soup.select('table', {'class': 'infobox vevent'})
                    wiki_table = wiki_tables[0] # get info table

                    # Loop through the tables until we have the info table we want
                    i = 0
                    while 'Directed by' not in wiki_table.text:
                        i += 1
                        wiki_table = wiki_tables[i]

                    # Some minor data cleaning
                    table_html = str(wiki_table).replace('<br/>', '/ ').replace('</li>', '/ ')
                    table_html = re.sub(r"\[\d+\]", "", table_html) # remove brackets (which provide link to references, but are not needed for our project)

                    # Use pd.read_html to create pandas dataframe of the info table
                    df = pd.read_html(table_html) 
                    df = pd.DataFrame(df[0]) # convert list to dataframe
                    df.columns = ['col_name', 'info']
                    df['col_name'] = df['col_name'].replace('/', ' ').replace('companies', 'company').replace('dates', 'date') # fix minor style error
                    df['info'] = df['info'].apply(lambda x: x[:-1] if str(x)[-1] == "/" else x) # fix minor style error

                    info_dict = {k: v for (k, v) in zip(list(df['col_name']), list(df['info'])) if k in info_list}
                    dict_to_append = {**film_dict, **info_dict}
                    
                # If the info table does not exist, then only append the film name, genre, and plot
                except:
                    dict_to_append = film_dict
                
                # Update the dataframe
                scraped_df = scraped_df.append(dict_to_append, ignore_index=True)

                
# Remove references from 'Plot' column              
scraped_df['Plot'] = scraped_df['Plot'].replace(r'\[\d+\]','', regex=True).replace(r'\[\w\]','', regex=True)           
scraped_df

ConnectTimeout: HTTPSConnectionPool(host='en.wikipedia.orghttps', port=443): Max retries exceeded with url: //en.wiktionary.org/wiki/gore (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x000001F6365569A0>, 'Connection to en.wikipedia.orghttps timed out. (connect timeout=5)'))

In [267]:
df4 = scraped_df
df4

,Film name,Genre,Plot,Directed by,Written by,Story by,Produced by,Starring,Cinematography,Edited by,Music by,Production company,Distributed by,Release date,Running time,Budget,Box office


In [286]:
df = pd.concat([df1, df2, df3], ignore_index=True)
df

,Film name,Genre,Plot,Directed by,Written by,Story by,Produced by,Starring,Cinematography,Edited by,Music by,Production company,Distributed by,Release date,Running time,Budget,Box office
0,The 2nd (film),Action,"While picking up his son at his college dorm, ...",Brian Skiba,Eric Bromberg/ Paul Taegel,NaN,Geoffrey James Clark/ Ryan Phillippe/ Kirk Sha...,Ryan Phillippe/ Casper Van Dien/ Jack Griffo/ ...,Adam Biddle,Scott Conrad,Richard Patrick,NaN,Momentum Pictures/ Voltage Pictures,"September 1, 2020",93 minutes,NaN,"$12,604"
1,3 Musketeers (film),Action,"Legendary black ops unit, The 3 Musketeers (ma...",Cole McKay,Edward DeRuiter,NaN,David Michael Latt/ David Rimawi/ Paul Bales,Heather Hemmens/ XIN/ Alan Rachins/ Keith Alla...,Ben Demaree,NaN,Chris Ridenhour,NaN,The Asylum,"October 25, 2011",90 minutes,"$300,000",NaN
2,10.0 Earthquake,Action,As a series of minor earthquakes start tearing...,David Gidali,J. Greg Abbott/ Alex Greenfield/ Nancy Leopardi,NaN,Steve Bencich/ Ross Kohn/ Nancy Leopardi,Henry Ian Cusick/ Jeffrey Jones,Damian Horan,Ruben Sebban,Leo J. Russlan,NaN,MarVista Entertainment,"October 7, 2014",87 minutes,NaN,NaN
3,12 Rounds (film),Action,A sting operation to capture arms dealer Miles...,Renny Harlin,Daniel Kunka,NaN,Becki Cross Trujillo/ Mark Gordon/ Renny Harli...,John Cena/ Aidan Gillen/ Ashley Scott/ Steve H...,David Boyd,Brian Berdan,Trevor Rabin,NaN,20th Century Fox,NaN,109 minutes,$6.7 million,$17.3 million
4,12 Rounds 2: Reloaded,Action,Paramedic Nick Malloy and his wife Sarah are g...,Roel Reiné,David Benullo,NaN,Michael J. Luisi,Randy Orton/ Tom Stevens/ Brian Markinson/ Ven...,Anthony C. Metchie,Radu Ion,Nathan Furst,NaN,20th Century Fox Home Entertainment,"June 4, 2013",94 minutes,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2986,The Yank,Satirical,"Irish American, Tom Murphy, is the first in hi...",Sean Lackey,Sean Lackey,NaN,NaN,Colm Meaney/ Fred Willard/ Kevin Farley/ Niki ...,Keith Nickoson,David Jingo,Chris Ainscough,NaN,Vision Films (US) / Bulldog Film Distribution ...,"March 23, 2014",NaN,NaN,NaN
2987,You Nazty Spy!,Satirical,The short begins with a title card disclaimer ...,Jules White,Felix Adler/ Clyde Bruckman,NaN,Jules White,Moe Howard/ Larry Fine/ Curly Howard/ Richard ...,NaN,NaN,NaN,NaN,Columbia Pictures,"January 19, 1940",18:00,NaN,NaN
2988,Zelig,Satirical,"Set in the 1920s and 1930s, the film concerns ...",Woody Allen,Woody Allen,NaN,Robert Greenhut,Woody Allen/ Mia Farrow,Gordon Willis,Susan E. Morse,Dick Hyman,NaN,Warner Bros.,"July 15, 1983",79 minutes,NaN,$11.8 million
2989,Zoolander,Satirical,"In New York City, male model Derek Zoolander i...",Ben Stiller,NaN,Drake Sather/ Ben Stiller,Scott Rudin/ Ben Stiller/ Stuart Cornfeld,Ben Stiller/ Owen Wilson/ Will Ferrell/ Christ...,Barry Peterson,Greg Hayden,David Arnold,NaN,Paramount Pictures,"September 28, 2001",90 minutes,$28 million,$60.8 million


In [337]:
df.at[2753, 'Running time'] = '18 minutes'
df.at[1277, 'Running time'] = '7 minutes'
df.at[1189, 'Running time'] = '6 minutes'
df.at[942, 'Running time'] = '1 minute'

In [344]:
# 'Duration (min)' extracts the number of minutes from 'Running time'
df['Duration (min)'] = df['Running time'].str.lower().str.extract(r'(?P<duration>[\d]+) min')

# 'Release year' extracts the year from 'Release date'
df['Release year'] = df['Release date'].str.extract(r'(\d{4})')

In [346]:
df.to_csv('Movie Genre and Plot.csv', index=False)

## Sentiment Analysis

In [343]:
# VADER Sentiment Analysis

from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

VADER_SentimentIntensityAnalyzer = SentimentIntensityAnalyzer()

def get_VADER_sentiment(text):
    return VADER_SentimentIntensityAnalyzer.polarity_scores(text)['compound']

df['VADER Sentiment'] = df['Plot'].apply(get_VADER_sentiment)

# Textblob Polarity and Subjectivity Analysis

from textblob import TextBlob

def get_TextBlob_subjectivity(text):
    return TextBlob(text).sentiment.subjectivity

def get_TextBlob_polarity(text):
    return TextBlob(text).sentiment.polarity

df['TextBlob Subjectivity'] = df['Plot'].apply(get_TextBlob_subjectivity)
df['TextBlob Polarity'] = df['Plot'].apply(get_TextBlob_polarity)

df.sort_values(by='TextBlob Polarity')

,Film name,Genre,Plot,Directed by,Written by,Story by,Produced by,Starring,Cinematography,Edited by,...,Production company,Distributed by,Release date,Running time,Budget,Box office,Duration (min),VADER Sentiment,TextBlob Subjectivity,TextBlob Polarity
1997,Disciples (film),Horror,Demons and humans band together to fight for t...,Joe Hollow,Joe Hollow,NaN,Wolfgang Meyer,Tony Todd/ Bill Moseley/ Angus Scrimm,Wolfgang Meyer,Joe Hollow,...,NaN,ITN Distribution,"September 12, 2014",90 minutes,NaN,NaN,90,-0.8979,1.00,-1.000000
1988,Death-Scort Service,Horror,The film follows a group of prostitutes workin...,Sean Donohue,Sean Donohue/ Chris Woods,NaN,Sean Donohue/ Shelby McIntyre/ John Miller/ Ch...,Krystal Pixie Adams/ Ashley Lynn Caputo/ Sean ...,Chris Woods,Chris Woods,...,NaN,NaN,"October 13, 2015",79 minutes,NaN,NaN,79,-0.9274,1.00,-0.875000
1689,Mind Games (1989 film),Thriller,"Rita and Dana Lund's marriage is in a crisis, ...",Bob Yari,Kenneth Dorward,NaN,Louie Lawless/ Randolf Turrow,Maxwell Caulfield/ Edward Albert/ Shawn Weathe...,Arnie Sirlin,Robert Gordon,...,NaN,Metro-Goldwyn-Mayer,"March 3, 1989",93 minutes,NaN,NaN,93,-0.8555,0.60,-0.787500
2024,Gates of Darkness,Horror,A dramatic mystery where a haunted teen endure...,Don E. Fauntleroy,Lesley-Anne Down/ Rob Hickman/ Chris Kanik/ Ra...,NaN,Don E. Fauntleroy/ Rob Hickman/ Laurence Lasca...,Tobin Bell,Don E. Fauntleroy,Bryan Colvin,...,NaN,NaN,October 2019,91 minutes,NaN,NaN,91,-0.7717,0.65,-0.608333
817,"Mobs, Inc.",Crime,Captain John Braddock schools a group of polic...,William Asher,NaN,NaN,NaN,Reed Hadley,NaN,NaN,...,NaN,NaN,"March 21, 1956",62 min.,NaN,NaN,62,-0.4767,0.90,-0.600000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1214,The Road to Freedom (film),War,"Two photojournalists, Sean (Joshua Fredric Smi...",Brendan Moriarty,NaN,NaN,Brendan Moriarty / Henry Bronson,Joshua Fredric Smith/ Scott Maguire,David Mun,Sean Halloran/ Margie Rogers,...,NaN,Creative Freedom,NaN,NaN,$1 million,NaN,NaN,0.5267,1.00,0.800000
2719,The Great American Beauty Contest,Satirical,"A feminist enters a beauty contest, hoping to ...",Robert Day,Stanford Whitmore,NaN,NaN,JoAnna Cameron/ Eleanor Parker/ Robert Cumming...,James A. Crabe,NaN,...,NaN,NaN,NaN,74 minutes,"$410,000",NaN,74,0.8860,0.40,0.800000
1727,Overexposed (film),Thriller,Morrison (Larry Brand) investigates when the c...,Larry Brand,NaN,NaN,Roger Corman,Catherine Oxenberg/ David Naughton/ Karen Black,NaN,NaN,...,NaN,NaN,1990,NaN,NaN,NaN,NaN,0.1280,1.00,0.850000
474,White Fang (1936 film),Action,"Based upon the legendary novel, a woman with h...",David Butler,NaN,NaN,Bogart Rogers/ Darryl F. Zanuck,Michael Whalen/ Jean Muir/ Slim Summerville/ C...,Arthur C. Miller,Irene Morra,...,NaN,20th Century Fox,"July 17, 1936",70 minutes,NaN,NaN,70,-0.9549,1.00,1.000000


## Categorizing emotion of text with the NRCLex library
https://medium.com/geekculture/simple-emotion-classification-in-python-40fb24692541

In [ ]:
!{sys.executable} -m pip install NRCLex

In [350]:
from nrclex import NRCLex
all_plots = ' '.join(df['Plot'])
text_object = NRCLex(all_plots)

ModuleNotFoundError: No module named 'nrclex'

## (Ignore) Testing Individual Links

In [249]:
# https://en.wikipedia.org/wiki/Yeti:_Curse_of_the_Snow_Demon 

link = 'https://en.wikipedia.org/wiki/Cherry_Bomb_(film)'
req = requests.get(link)
soup = BeautifulSoup(req.content, 'html')
wiki_tables = soup.select('table', {'class': 'infobox vevent'})
wiki_table = wiki_tables[0] # get info table

# loop through the tables until we have the info table we want
i = 0
while 'Directed by' not in wiki_table.text:
    i += 1
    wiki_table = wiki_tables[i]

# Data cleaning
table_html = str(wiki_table).replace('<br/>', '/ ').replace('</li>', '/ ')
table_html = re.sub(r"\[\d+\]", "", table_html) # remove brackets (which provide link to references, but are not needed for our project)

# Use pd.read_html to create pandas dataframe of the info table
df = pd.read_html(table_html) 
df = pd.DataFrame(df[0]) # convert list to dataframe
df.columns = ['col_name', 'info']
df['col_name'] = df['col_name'].replace('/', ' ').replace('companies', 'company').replace('dates', 'date') # fix minor style error
df['info'] = df['info'].apply(lambda x: x[:-1] if str(x)[-1] == "/" else x) # fix minor style error
display(df)

info_dict = {k: v for (k, v) in zip(list(df['col_name']), list(df['info'])) if k in info_list}

info_dict

,col_name,info
0,DVD cover,DVD cover
1,Directed by,Kyle Day
2,Screenplay by,Garrett Hargrove
3,Story by,Kyle Day
4,Produced by,Kyle Day/ Garrett Hargrove/ Jason Latimer
5,Starring,Julin Jean/ Nick Manning/ John Gabriel Rodrigu...
6,Cinematography,Andrew Michael Barrera
7,Edited by,Kyle Day/ David Ward
8,Production/ company,Strike Anywhere Productions
9,Distributed by,Well Go USA


{'Directed by': 'Kyle Day',
 'Story by': 'Kyle Day',
 'Produced by': 'Kyle Day/ Garrett Hargrove/ Jason Latimer',
 'Starring': 'Julin Jean/ Nick Manning/ John Gabriel Rodriguez/ Allen Hackley/ Jeremy James Douglas Norton/ Aaron Alexander',
 'Cinematography': 'Andrew Michael Barrera',
 'Edited by': 'Kyle Day/ David Ward',
 'Distributed by': 'Well Go USA',
 'Running time': '82 minutes'}